This lab on Splines and GAMs in R comes from p. 293-297 of "Introduction to Statistical Learning with Applications in R" by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani.

In [ ]:
library(ISLR)

# 7.8.2 Splines

In order to fit regression splines in R, we use the ${\tt splines}$ library. In lecture, we saw that regression splines can be fit by constructing an appropriate matrix of basis functions. The ${\tt bs()}$ function generates the entire matrix of basis functions for splines with the specified set of knots. By default, cubic
splines are produced. Fitting wage to age using a regression spline is simple:

In [ ]:
library(splines)

# Get min/max values of age
agelims=range(Wage$age)

# Generate a sequence of age values spanning the range
age.grid=seq(from=agelims[1],to=agelims[2])

# Fit a regression spline using basis functions
fit=lm(wage~bs(age,knots=c(25,40,60)),data=Wage)

# Predict the value of the generated ages, returning the standard error using se=TRUE
pred=predict(fit,newdata=list(age=age.grid),se=T)

# Plot the spline and error bands
plot(age,wage,col="gray")
lines(age.grid,pred$fit, col="red", lwd=2)
lines(age.grid,pred$fit+2*pred$se,lty="dashed")
lines(age.grid,pred$fit-2*pred$se,lty="dashed")

Here we have prespecified knots at ages 25, 40, and 60. This produces a
spline with six basis functions. (Recall that a cubic spline with three knots
has seven degrees of freedom; these degrees of freedom are used up by an
intercept, plus six basis functions.) We could also use the ${\tt df}$ option to
produce a spline with knots at uniform quantiles of the data:

In [ ]:
# Specifying knots directly: 6 basis functions
dim(bs(age,knots=c(25,40,60)))

# Specify desired degrees of freedom, select knots automatically: still 6 basis functions
dim(bs(age,df=6))

# Show me where the knots were placed
attr(bs(age,df=6),"knots")

In this case R chooses knots at ages 33.8, 42.0, and 51.0, which correspond
to the 25th, 50th, and 75th percentiles of age. The function ${\tt bs()}$ also has
a ${\tt degree}$ argument, so we can fit splines of any degree, rather than the
default degree of 3 (which yields a cubic spline).

In order to instead fit a natural spline, we use the ${\tt ns()}$ function. Here
we fit a natural spline with four degrees of freedom:

In [ ]:
fit2=lm(wage~ns(age,df=4),data=Wage)
pred2=predict(fit2,newdata=list(age=age.grid),se=T)

# Plot the natural spline and error bands
plot(age,wage,col="gray")
lines(age.grid, pred2$fit, col="red",lwd=2)
lines(age.grid,pred2$fit+2*pred$se,lty="dashed")
lines(age.grid,pred2$fit-2*pred$se,lty="dashed")

As with the ${\tt bs()}$ function, we could instead specify the knots directly using
the ${\tt knots}$ option.

In order to fit a smoothing spline, we use the ${\tt smooth.spline()}$ function. Here we'll reproduce the plot we saw in lecture showing a 16-degree and LOOCV smoothing spline on the ${\tt Wage}$ data:

In [ ]:
plot(age,wage,xlim=agelims,cex=.5,col="darkgrey")
title("Smoothing Spline")
fit=smooth.spline(age,wage,df=16)
fit2=smooth.spline(age,wage,cv=TRUE)
lines(fit,col="red",lwd=2)
lines(fit2,col="blue",lwd=2)
legend("topright",legend=c("16 Degrees of Freedom","6.8 Effective Degrees of Freedom (CV)"),col=c("red","blue"),lty=1,lwd=2,cex=.8)

Notice that in the first call to ${\tt smooth.spline()}$, we specified ${\tt df=16}$. The
function then determines which value of $\lambda$ leads to 16 degrees of freedom. In
the second call to ${\tt smooth.spline()}$, we select the smoothness level by cross-validation;
this results in a value of $\lambda$ that yields 6.8 degrees of freedom.

# 7.8.3 GAMs
We now fit a really simple GAM to predict wage using natural spline functions of ${\tt year}$
and ${\tt age}$, treating ${\tt education}$ as a qualitative predictor. Since
this is just a big linear regression model using an appropriate choice of
basis functions, we can simply do this using the ${\tt lm()}$ function:

In [ ]:
gam1=lm(wage~ns(year,4)+ns(age,5)+education,data=Wage)

What if we want to fit the model using smoothing splines rather than natural
splines? In order to fit more general sorts of GAMs, using smoothing splines
or other components that cannot be expressed in terms of basis functions
and then fit using least squares regression, we will need to use the ${\tt gam}$
library in R. The ${\tt s()}$ function, which is part of the ${\tt gam}$ library, is used to indicate that
we would like to use a smoothing spline. We'll specify that the function of
${\tt year}$ should have 4 degrees of freedom, and that the function of ${\tt age}$ will
have 5 degrees of freedom. Since ${\tt education}$ is qualitative, we leave it as is,
and it is converted into four dummy variables.

We can use the ${\tt gam()}$ function in order to fit a GAM using these components. All of the terms are
fit simultaneously, taking each other into account to explain the response:

In [ ]:
library(gam)
gam.m3=gam(wage~s(year,4)+s(age,5)+education,data=Wage)
par(mfrow=c(1,3))
plot(gam.m3, se=TRUE,col="blue")

The generic ${\tt plot()}$ function recognizes that gam2 is an object of class gam,
and invokes the appropriate ${\tt plot.gam()}$ method. Conveniently, even though our simple
${\tt gam1}$ is not of class ${\tt gam}$ but rather of class ${\tt lm}$, we can still use ${\tt plot.gam()}$
on it:

In [ ]:
par(mfrow=c(1,3))
plot.gam(gam1, se=TRUE, col="red")

Notice here we had to use ${\tt plot.gam()}$ rather than the generic ${\tt plot()}$
function.

In these plots, the function of ${\tt year}$ looks rather linear. We can perform a
series of ANOVA tests in order to determine which of these three models is
best: a GAM that excludes ${\tt year}$ ($M_1$), a GAM that uses a linear function
of ${\tt year}$ ($M_2$), or a GAM that uses a spline function of ${\tt year}$ ($M_3$):

In [ ]:
gam.m1=gam(wage~s(age,5)+education,data=Wage)
gam.m2=gam(wage~year+s(age,5)+education,data=Wage)
print(anova(gam.m1,gam.m2,gam.m3,test="F"))

We find that there is compelling evidence that a GAM with a linear function
of ${\tt year}$ is better than a GAM that does not include ${\tt year}$ at all
($p$-value=0.00014). However, there is no evidence that a non-linear function
of ${\tt year}$ is helpful ($p$-value=0.349). In other words, based on the results
of this ANOVA, $M_2$ is preferred.

The ${\tt summary()}$ function produces a summary of the GAM fit:

In [ ]:
summary(gam.m3)

The $p$-values for ${\tt year}$ and ${\tt age}$ correspond to a null hypothesis of a linear
relationship versus the alternative of a non-linear relationship. The large
$p$-value for ${\tt year}$ reinforces our conclusion from the ANOVA test that a linear
function is adequate for this term. However, there is very clear evidence
that a non-linear term is required for ${\tt age}$.

We can make predictions from ${\tt gam}$ objects, just like from ${\tt lm}$ objects,
using the ${\tt predict()}$ method for the class ${\tt gam}$. Here we make predictions on
the training set:

In [ ]:
preds=predict(gam.m2,newdata=Wage)

# Logistic Regression GAMs
In order to fit a logistic regression GAM, we once again use the ${\tt I()}$ function
in constructing the binary response variable, and set ${\tt family=binomial}$:

In [ ]:
gam.lr=gam(I(wage>250)~year+s(age,df=5)+education,family=binomial,data=Wage)
par(mfrow=c(1,3))
plot(gam.lr,se=T,col="green")

It is easy to see that there are no high earners in the ${\tt <HS}$ category:

In [ ]:
table(education,I(wage>250))

Hence, we fit a logistic regression GAM using all but this category. This
provides more sensible results:

In [ ]:
gam.lr.s=gam(I(wage>250)~year+s(age,df=5)+education,family=binomial,data=Wage,subset=(education!="1. < HS Grad"))
par(mfrow=c(1,3))
plot(gam.lr.s,se=T,col="green")

To get credit for this lab, post your answer to thew following question:
 - How would you choose whether to use a polynomial, step, or spline function for each predictor when building a GAM?

To Piazza: https://piazza.com/class/igwiv4w3ctb6rg?cid=50